# **Section I: From data to singletons**

In [ ]:
#Importing data
#With google docs
import csv
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
path ='drive/My Drive/Colab Notebooks/data/T10I4D100K.dat'
''' path = WITHOUT COLAB, define path here '''
df = pd. read_csv(path, sep=',', names=['Transactions'])

Mounted at /content/drive


In [ ]:
df

,Transactions
0,25 52 164 240 274 328 368 448 538 561 630 687 ...
1,39 120 124 205 401 581 704 814 825 834
2,35 249 674 712 733 759 854 950
3,39 422 449 704 825 857 895 937 954 964
4,15 229 262 283 294 352 381 708 738 766 853 883...
...,...
99995,39 132 623 704 795 825 853
99996,72 120 241 268 351 354 545 567 606 749 841 893
99997,82 313 406 421 438 629
99998,69 75 140 175 285 494 598 614 641 723 829 871


In [ ]:
transactions = list(df['Transactions'])

In [ ]:
len(transactions)

100000

In [ ]:
for i in range(0,len(transactions)):
  transactions[i] = transactions[i].split()
  transactions[i] = list(map(int, transactions[i]))

In [ ]:
def get_singletons(transactions):
  singletons  = dict()
  for transaction in transactions:
    for item in transaction:
      if item not in singletons:
        singletons[item] = 1
      else:
        singletons[item] = singletons[item] + 1
  return singletons


singletons = get_singletons(transactions)
print(len(singletons))

870


In [ ]:
def filter_itons(itons, support_threshold):
  itons_filtered = {}
  for key, value in itons.items():
      if value >= support_threshold:
        itons_filtered[key] = value
  return dict(sorted(itons_filtered.items(),key=lambda x:x[0]))

singletons_filtered = filter_itons(singletons, 3500)
len(singletons_filtered)

40

### **A Priori Algorithm**

#### From singleton to doubletons

In [ ]:
#create candidate pairs
def getDoubletonsCandidates(singletons):
  return combinations(singletons, 2)

candidate_pairs_doubletons = getDoubletonsCandidates(singletons_filtered)

In [ ]:
doubletons = dict()
#calculate support for candidate  pairs
for doubleton in candidate_pairs_doubletons:
  for transaction in transactions:
    if doubleton[0] in transaction and doubleton[1] in transaction:
        if doubleton not in doubletons:
          doubletons[doubleton] = 1
        else:
          doubletons[doubleton] =  doubletons[doubleton] + 1
print(doubletons)

{(32, 39): 121, (32, 71): 117, (32, 120): 144, (32, 145): 246, (32, 177): 142, (32, 183): 114, (32, 205): 150, (32, 217): 254, (32, 283): 112, (32, 354): 222, (32, 362): 111, (32, 368): 248, (32, 401): 303, (32, 419): 154, (32, 438): 127, (32, 460): 283, (32, 470): 243, (32, 494): 313, (32, 529): 224, (32, 538): 115, (32, 541): 116, (32, 682): 123, (32, 684): 164, (32, 692): 163, (32, 720): 414, (32, 722): 191, (32, 766): 174, (32, 775): 144, (32, 789): 131, (32, 829): 216, (32, 862): 107, (32, 883): 192, (32, 888): 109, (32, 914): 262, (32, 919): 126, (32, 937): 453, (32, 947): 720, (32, 956): 111, (32, 966): 364, (39, 71): 104, (39, 120): 165, (39, 145): 734, (39, 177): 148, (39, 183): 111, (39, 205): 107, (39, 217): 172, (39, 283): 356, (39, 354): 310, (39, 362): 147, (39, 368): 565, (39, 401): 112, (39, 419): 511, (39, 438): 215, (39, 460): 147, (39, 470): 468, (39, 494): 144, (39, 529): 232, (39, 538): 108, (39, 541): 101, (39, 682): 120, (39, 684): 152, (39, 692): 155, (39, 720):

In [ ]:
#filter doubletons
doubletons_filtered = filter_itons(doubletons,300)
print(len(doubletons_filtered))

151


#### From doubletons to tripletons

In [ ]:
from itertools import combinations

def getCandidatePairs(doubletons_filtered):
  flatList = []
  tripletonCandidate = []
  for row in doubletons_filtered: flatList.extend(row)
  flatList = combinations(set(flatList),3)
  flatList = [value for value in flatList]
  print(len(flatList))
  for tripleton in flatList:
    count = 0
    tuple1 = set((tripleton[0],tripleton[1]))
    tuple2 = set((tripleton[0],tripleton[2]))
    tuple3 = set((tripleton[1],tripleton[2]))
    for doubleton in doubletons_filtered:
      if len(tuple1.intersection(doubleton)) == 2 or len(tuple2.intersection(doubleton)) == 2 or len(tuple3.intersection(doubleton)) == 2:
        count +=1
    if count == 3: tripletonCandidate.append(tripleton)
  return tripletonCandidate

candidateTripletons = getCandidatePairs(doubletons_filtered)

9880


In [ ]:
%%time
threshhold = 300
tripletons = dict()

for value in candidateTripletons:
  for transaction in transactions:
    l = set(value).intersection(transaction)
    if len(l) == 3:
      #print("yes")
      if value not in tripletons:
        tripletons[value] = 1
      else:
        tripletons[value] = tripletons[value] + 1
print(len(tripletons),tripletons)

140 {(401, 684, 460): 44, (401, 460, 205): 31, (401, 205, 883): 208, (145, 283, 39): 265, (145, 283, 217): 38, (145, 419, 39): 352, (145, 419, 217): 37, (145, 419, 368): 42, (145, 39, 354): 24, (145, 39, 368): 56, (145, 937, 682): 213, (145, 937, 494): 210, (145, 937, 368): 215, (145, 682, 494): 220, (145, 682, 368): 232, (145, 217, 368): 19, (145, 354, 368): 20, (145, 494, 368): 219, (529, 538, 937): 26, (529, 538, 460): 25, (529, 538, 205): 23, (529, 538, 368): 33, (529, 541, 937): 254, (529, 541, 947): 243, (529, 937, 947): 243, (529, 937, 71): 34, (529, 937, 368): 47, (529, 684, 438): 36, (529, 684, 829): 22, (529, 684, 71): 32, (529, 684, 460): 44, (529, 684, 368): 33, (529, 684, 766): 40, (529, 947, 217): 38, (529, 947, 368): 30, (529, 438, 71): 21, (529, 438, 368): 37, (529, 438, 766): 21, (529, 829, 205): 239, (529, 829, 368): 75, (529, 829, 766): 21, (529, 71, 354): 21, (529, 71, 766): 37, (529, 460, 205): 31, (529, 460, 368): 41, (529, 205, 368): 31, (529, 205, 883): 79, (529

In [ ]:
tripletons_filtered = filter_itons(tripletons,300)
print(len(tripletons_filtered))

3


### Old Version --> Solved through possible combinations but not a priori
First steps can be done in seconds for this dataset at every threshold

In [ ]:
singletons = get_singletons(transactions)
singletons_filtered = filter_itons(singletons, 500)

In [ ]:
%%time
threshhold = 500
doubletons = dict()
for transaction in transactions:
  # combinations
  # this works, since every transcation is ordered:
  for idx_item1 in range(len(transaction)):
    for idx_item2 in range(idx_item1+1,len(transaction)):
      #check frequency and prune
      #if the items are not frequent, they are considerd not to be a candidate pair
      if singletons[transaction[idx_item1]] > threshhold and singletons[transaction[idx_item2]] > threshhold:
        #create doubleton
        doubleton = (transaction[idx_item1],transaction[idx_item2])
        #Create new entry in doubletons dictionary
        if doubleton not in doubletons:
          doubletons[doubleton] = 1
        #If already exists, increment by 1
        else:
          doubletons[doubleton] =  doubletons[doubleton] + 1


CPU times: user 7.56 s, sys: 19.9 ms, total: 7.58 s
Wall time: 7.69 s


In [ ]:
doubletons_filtered = filter_itons(doubletons,500)

### Tripletons

In [ ]:
%%time
threshhold = 500
tripletons = dict()
for transaction in transactions:
  #print(transaction)
  # combinations
  # this works, since every transcation is ordered:
  for idx_item1 in range(len(transaction)):
    start_number = transaction[idx_item1]
    for idx_item2 in range(idx_item1+1,len(transaction)):
      for idx_item3 in range(idx_item2+1, len(transaction)):
        #check frequency and prune
        #if the items are not frequent, they are considerd not to be a candidate pair
        #print(transaction,idx_item2)
        #print(transaction[idx_item1], transaction[idx_item2])
        double1 = tuple((transaction[idx_item1], transaction[idx_item2]))
        double2 = tuple((transaction[idx_item1], transaction[idx_item3]))
        double3 = tuple((transaction[idx_item2], transaction[idx_item3]))
        #print(start_number,double)
        #print("Item1",transaction[idx_item1],"item2",transaction[idx_item2:idx_item2+2])
        #print(double)
        if double1 in doubletons_filtered and double2 in doubletons_filtered and double3 in doubletons_filtered:
          tripleton = (start_number,transaction[idx_item2],transaction[idx_item3])
          #print(transaction, tripleton)
          if tripleton not in tripletons:
            tripletons[tripleton] = 1
          else:
            tripletons[tripleton] = tripletons[tripleton] + 1

print(len(tripletons),tripletons)

170 {(39, 704, 825): 1035, (708, 883, 978): 542, (571, 623, 795): 704, (571, 623, 853): 713, (571, 795, 853): 715, (597, 766, 960): 441, (623, 795, 853): 720, (392, 461, 569): 459, (392, 461, 862): 466, (392, 569, 801): 470, (392, 569, 862): 475, (392, 801, 862): 637, (461, 569, 862): 462, (569, 801, 862): 472, (350, 411, 572): 565, (350, 411, 579): 562, (350, 411, 803): 573, (350, 411, 842): 566, (350, 572, 579): 557, (350, 572, 803): 563, (350, 572, 842): 561, (350, 579, 803): 565, (350, 579, 842): 562, (350, 803, 842): 570, (411, 572, 579): 560, (411, 572, 803): 567, (411, 572, 842): 561, (411, 579, 803): 573, (411, 579, 842): 568, (411, 803, 842): 575, (572, 579, 803): 559, (572, 579, 842): 558, (572, 803, 842): 562, (579, 803, 842): 570, (208, 290, 458): 751, (208, 290, 888): 764, (208, 290, 969): 764, (208, 458, 888): 757, (208, 458, 969): 755, (208, 888, 969): 771, (290, 458, 888): 750, (290, 458, 969): 748, (290, 888, 969): 765, (458, 888, 969): 756, (192, 487, 638): 495, (192,

In [ ]:
tripletons_filtered = filter_itons(tripletons,500)
print(len(tripletons_filtered))

110


# **II Section : Calculate Confidence**

In [ ]:
prueba_tripletons = dict()
for key,value in tripletons_filtered.items():
  prueba_tripletons[frozenset(key)] = value
prueba_doubletons = dict()
for key,value in doubletons_filtered.items():
  prueba_doubletons[frozenset(key)] = value


In [ ]:
def get_association_rules_doubletons(conf_level):
  association_rules_doubletons = dict()
  for doubleton in doubletons_filtered.keys():
    item1 = doubleton[0]
    item2 = doubleton[1]
    support_union = doubletons_filtered[doubleton]
    support_item1 = singletons_filtered[item1]
    support_item2 = singletons_filtered[item2]
    confidence1 = support_union/support_item1
    confidence2 = support_union/support_item2
    if confidence1 > conf_level:
      association_rules_doubletons[item1] = item2
    if confidence2 > conf_level:
      association_rules_doubletons[item2] = item1
  return association_rules_doubletons

association_rules_doubletons = get_association_rules_doubletons(0.8)
print(association_rules_doubletons)

{969: 888, 842: 803, 626: 496, 801: 862}


In [ ]:
def get_association_rules_tripletons1to2(conf_level):
  association_rules_tripletons = dict()
  for tripleton in tripletons_filtered.keys():
    support_union = tripletons_filtered[tripleton]
    item1 = tripleton[0]
    item2 = tripleton[1]
    item3 = tripleton[2]
    support_item1 = singletons_filtered[item1]
    support_item2 = singletons_filtered[item2]
    support_item3 = singletons_filtered[item3]

    # Rule1: Item1 --> (Item2, Item3)
    confidence_rule1 = support_union/support_item1
    if confidence_rule1 > conf_level:
      association_rules_tripletons[item1] = [item2,item3]

    # Rule2: Item2 --> (Item1, Item3)
    confidence_rule2 = support_union/support_item2
    if confidence_rule2 > conf_level:
      association_rules_tripletons[item2] = [item1,item3]

    # Rule3: Item3 --> (Item1, Item2)
    confidence_rule3 = support_union/support_item3
    if confidence_rule1 > conf_level:
      association_rules_tripletons[item3] = [item1,item2]


  return association_rules_tripletons

association_rules_tripletons_1to2 = get_association_rules_tripletons1to2(0.4)
print(association_rules_tripletons_1to2)

{33: [346, 515], 283: [33, 217], 346: [33, 283], 515: [33, 346], 704: [39, 825], 765: [684, 819], 105: [862, 944], 815: [862, 944], 862: [105, 815], 944: [815, 862], 158: [583, 617], 583: [354, 617], 617: [158, 583], 185: [678, 960], 678: [471, 960], 960: [185, 678], 208: [888, 969], 290: [888, 969], 458: [888, 969], 888: [290, 458], 969: [458, 888], 546: [923, 947], 227: [390, 722], 722: [227, 390], 801: [392, 862], 534: [470, 995], 923: [546, 661], 947: [546, 923], 708: [883, 978], 978: [708, 883]}


In [ ]:
def get_association_rules_tripletons2to1(conf_level):
  association_rules_tripletons_2to1 = dict()
  for tripleton in tripletons_filtered.keys():
    support_union = tripletons_filtered[tripleton]
    item1 = tripleton[0]
    item2 = tripleton[1]
    item3 = tripleton[2]
    support_item1 = singletons_filtered[item1]
    support_item2 = singletons_filtered[item2]
    support_item3 = singletons_filtered[item3]

    # Rule1: (Item1, Item2) --> Item3
    key1 = frozenset([item1,item2])
    confidence_rule1 = support_union/ prueba_doubletons[key1]
    if confidence_rule1 > conf_level:
      association_rules_tripletons_2to1[key1] = item3

    # Rule2: (Item1, Item3) --> Item2
    key2 = frozenset([item1,item3])
    confidence_rule2 = support_union/ prueba_doubletons[key2]
    if confidence_rule2 > conf_level:
      association_rules_tripletons_2to1[key2] = item2

    # Rule3: (Item2, Item3) --> Item1
    key3 = frozenset([item2,item3])
    confidence_rule3 = support_union/ prueba_doubletons[key3]
    if confidence_rule3 > conf_level:
      association_rules_tripletons_2to1[key3] = item1

  return association_rules_tripletons_2to1

association_rules_tripletons_2to1 =  get_association_rules_tripletons2to1(0.8)
print(association_rules_tripletons_2to1)
print(len(association_rules_tripletons_2to1))


{frozenset({4, 373}): 716, frozenset({716, 4}): 373, frozenset({716, 373}): 4, frozenset({32, 6}): 472, frozenset({472, 6}): 32, frozenset({32, 472}): 6, frozenset({227, 12}): 722, frozenset({12, 390}): 722, frozenset({722, 12}): 390, frozenset({21, 413}): 857, frozenset({793, 21}): 857, frozenset({793, 413}): 857, frozenset({857, 21}): 793, frozenset({857, 413}): 793, frozenset({857, 793}): 413, frozenset({32, 285}): 947, frozenset({32, 947}): 285, frozenset({947, 285}): 32, frozenset({33, 217}): 515, frozenset({33, 283}): 515, frozenset({217, 283}): 515, frozenset({33, 346}): 515, frozenset({33, 515}): 346, frozenset({217, 515}): 346, frozenset({346, 283}): 515, frozenset({515, 283}): 346, frozenset({346, 515}): 283, frozenset({704, 39}): 825, frozenset({825, 39}): 704, frozenset({704, 825}): 39, frozenset({944, 54}): 998, frozenset({54, 998}): 944, frozenset({944, 998}): 54, frozenset({684, 70}): 819, frozenset({765, 70}): 819, frozenset({684, 765}): 819, frozenset({819, 70}): 765, 